In [7]:
import requests

In [8]:
import json
import concurrent.futures
import threading
import time
import logging
import os
from tqdm import tqdm
import datetime
import pandas as pd
import csv
import subprocess

## Kaggle Secrets


In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")
tmdb_key = user_secrets.get_secret("tmdb_key")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

ModuleNotFoundError: No module named 'kaggle_secrets'

## Basic file configurations for the project


In [11]:
# Get current working directory
cwd = os.getcwd()

# Define folder paths
data_folder = os.path.join(cwd, 'TMDB_movie_data')
archive_folder = os.path.join(data_folder, 'TMDB_archive')
output_folder = os.path.join(data_folder, 'temp_movies')
log_file_path = os.path.join(data_folder, 'logs')
saved_movie_tracker_path = os.path.join(
    archive_folder, 'completed_movie_ids.txt')

# Create necessary folders if they don't exist
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)
os.makedirs(log_file_path, exist_ok=True)
os.makedirs(archive_folder, exist_ok=True)

# Get current date
date_today = datetime.datetime.utcnow().date()
date_today_str = str(date_today)

# Create file path for saving json file
json_save_file_path = os.path.join(
    archive_folder, f'{date_today_str}_TMDB_movies.ndjson')

/var/folders/5b/p6kdbqhn57dbt096gwlqtkjh0000gn/T/ipykernel_70412/1483241695.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_today = datetime.datetime.utcnow().date()


## Kaggle File Paths


In [24]:
# folder path for Kaggle files
kaggle_folder = os.path.join(data_folder, 'to_kaggle')
os.makedirs(kaggle_folder, exist_ok=True)

# folder path for Kaggle dataset zip files
kaggle_zip_folder = os.path.join(kaggle_folder, 'TMDB_movie_dataset_v11')
os.makedirs(kaggle_zip_folder, exist_ok=True)

# file path for Kaggle dataset CSV file
kaggle_file = os.path.join(
    kaggle_zip_folder, 'TMDB_movie_dataset_v11.csv')

## Global variables


In [25]:
# variable to store the number of movies collected
movies_collected = 0
lock = threading.Lock()
dataset_df = pd.read_csv('/kaggle/input/tmdb-movies-dataset-2023-930k-movies/TMDB_movie_dataset_v11.csv')
full_log = False # Mark True to log everything

## Setup logger


In [26]:
def setup_logger() -> logging:
    # Create a logger for the current module
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Define the log format
    log_formatter = logging.Formatter(
        '%(asctime)s    | %(name)s  | %(levelname)s | %(message)s')

    # Create a stream handler to output log messages to the console
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(log_formatter)
    logger.addHandler(stream_handler)

    return logger

## Get the latest movie id


In [27]:
def get_latest() -> int:

    url = "https://api.themoviedb.org/3/movie/latest"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {tmdb_key}"
    }

    # Retry the request until it succeeds
    retries = 0
    while retries < 500:
        try:
            response = requests.get(url, headers=headers)
            break
        except:
            retries += 1
            time.sleep(5)

    # Extract the ID of the latest movie from the response
    response_in_json = json.loads(response.text)
    movies_id = int(response_in_json['id'])

    return movies_id

## Get the last movie id from my dataset


In [28]:
def get_oldest() -> int:
    global dataset_df
    
    # Get the 'id' value of the last row in the sorted DataFrame
    dataset_df.sort_values(['id'],inplace=True)
    last_id = dataset_df.iloc[-1]['id']
    
    return last_id

## Delete old temp files.

Not needed on kaggle, but when running locally, this is a must


In [29]:
def delete_old_files(logger: logging) -> None:
    logger.info("Deleting Old temp files")
    try:
        # Get a list of files from the last run in the output folder
        files_from_last_run = os.listdir(output_folder)
        
        # delete old temp files
        if len(files_from_last_run) > 0:
            for file in files_from_last_run:
                full_file_path = os.path.join(output_folder, file)
                os.remove(full_file_path)
    except Exception as error:
        logger.error(f"Error when deleting old temp files: {error}")

## Main function that handles getting movie data


In [30]:
def get_keywords(movie_id: int) -> dict | None:
    """
    Gets keywords from TMDB
    """

    url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords"

    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {tmdb_key}",
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.text
    return None

In [31]:
def parse_keywords(keywords: str) -> str:
    """
    Parses the keywords into a list
    """

    keywords_dict = json.loads(keywords)

    keywords_list = []

    for item in keywords_dict["keywords"]:
        keywords_list.append(item["name"])

    return ", ".join(keywords_list)

In [32]:
def handler_get_keywords(movie_id: int) -> list[int, str]:
    """
    handles getting keywords
    """

    data = get_keywords(movie_id)

    if not data:
        return None

    keywords_str = parse_keywords(data)

    return keywords_str

In [33]:
def process_movie_ids(movie_id: int, logger: logging, pbar: tqdm) -> None:
    global movies_collected
    
    if full_log:
        logger.info(f'Processing {movie_id}')
    
    # output file path for the current movie
    output_file = os.path.join(
        output_folder, f'scrapeTMDB_movies_{movie_id}.ndjson')
    with lock:
        pbar.update(1)

    try:
        time.sleep(0.2)
        url = f"https://api.themoviedb.org/3/movie/{movie_id}?language=en-US"
        headers = {
            "accept": "application/json",
            "Authorization": f"Bearer {tmdb_key}"
        }
        response = requests.get(url, headers=headers)
        
        if response.status_code >= 500 or response.status_code == 429:
            logger.warning(
                f"movie {movie_id} - Got {response.status_code} response, retrying...")
            time.sleep(1)

        # Process the response if it is successful
        if response.status_code == 200:
            rqst_json = response.json()
            
            keywords = handler_get_keywords(movie_id)
            rqst_json["keywords"] = keywords
            with open(output_file, 'a', encoding="utf-8") as append_file:
                append_file.write(json.dumps(rqst_json) + '\n')

            # Increment the count of collected movies and save the movie ID as completed
            # this way, even if the code fails, we can restart where we left off
            # Not needed for daily updates, but if running from 0, this will be userful
            with lock:
                movies_collected += 1
                with open(saved_movie_tracker_path, 'a', encoding='utf-8') as completed_movie_id_file:
                    completed_movie_id_file.write(f'{movie_id}\n')

            if full_log:
                logger.info(f'movies collected: {movies_collected}')

    except Exception as error:
        logger.error(
            f'Expection for {movie_id} in process_movie_ids: {error}')
            
    if full_log:
        logger.info(f'Completed movie {movie_id}')

## function to combine all the threads into 1 ndjson file


In [34]:
def combine_threads(logger: logging) -> None:
    logger.info('Starting to combine threads')
    thread_dir = output_folder

    count = 0
    file_names = os.listdir(thread_dir)

    # open the save file
    with open(json_save_file_path, 'a', encoding='utf-8') as append_file:
        for file in file_names:
            read_file_path = os.path.join(thread_dir, file)
            with open(read_file_path, 'r', encoding='utf-8') as read_file:
                
                for line in read_file:
                    line_data = json.loads(line)
                    count += 1
                    # save the line to the main json
                    append_file.write(f'{json.dumps(line_data)}\n')

    logger.info(f'Finished combining threads. Total movies added: {count}')

## load the ndjson file and make a python list from the data


In [35]:
def load_json_data() -> list[dict]:
    """This function will take the combined thread ndjson file, parse it, format it, and return all the movies as python list of dicts"""
    
    data = []
    unique_ids = set()

    # load the data
    with open(json_save_file_path, 'r', encoding='utf-8') as input_file:
        variations_to_ignore = [
            None, "", "NA", "N/A", "None", "na", "n/a", "NULL", "Not Available"]

        for line in input_file:
            line_in_json = json.loads(line)

            def format_list_to_str(line_in_json: dict, formatted_data: dict) -> dict:
                # convert genres list to str
                
                genres_str = None
                genres_list = []
                try:
                    for genre in line_in_json['genres']:
                        if genre['name'] is not any(variations_to_ignore):
                            genres_list.append(genre['name'])
                    if len(genres_list) > 0:
                        genres_str = ', '.join(genres_list)
                except:
                    pass

                formatted_data['genres'] = genres_str

                # convert production_companies list to str
                production_companies_str = None
                production_companies_list = []
                try:
                    for company in line_in_json['production_companies']:
                        if company['name'] is not any(variations_to_ignore):
                            production_companies_list.append(company['name'])
                    if len(production_companies_list) > 0:
                        production_companies_str = ', '.join(
                            production_companies_list)
                except:
                    pass

                formatted_data['production_companies'] = production_companies_str

                # convert production_countries list to str
                production_countries_str = None
                production_countries_list = []
                try:
                    for country in line_in_json['production_countries']:
                        if country['name'] is not any(variations_to_ignore):
                            production_countries_list.append(country['name'])
                    if len(production_countries_list) > 0:
                        production_countries_str = ', '.join(
                            production_countries_list)
                except:
                    pass

                formatted_data['production_countries'] = production_countries_str

                # convert spoken_languages list to str
                spoken_languages_str = None
                spoken_languages_list = []
                try:
                    for language in line_in_json['spoken_languages']:
                        if language['english_name'] is not any(variations_to_ignore):
                            spoken_languages_list.append(
                                language['english_name'])
                    if len(spoken_languages_list) > 0:
                        spoken_languages_str = ', '.join(spoken_languages_list)
                except:
                    pass

                formatted_data['spoken_languages'] = spoken_languages_str

                return formatted_data

            # format the data

            formatted_data = {}

            formatted_data["id"] = int(
                line_in_json['id']) if line_in_json['id'] is not None else 0

            formatted_data["title"] = str(
                line_in_json['title']) if line_in_json['title'] is not any(variations_to_ignore) else None
            formatted_data["vote_average"] = float(
                line_in_json['vote_average']) if line_in_json['vote_average'] is not None else 0.0

            formatted_data["vote_count"] = int(
                line_in_json['vote_count']) if line_in_json['vote_count'] is not None else 0

            formatted_data["status"] = str(
                line_in_json['status']) if line_in_json['status'] is not any(variations_to_ignore) else None

            formatted_data["release_date"] = str(
                line_in_json['release_date']) if line_in_json['release_date'] is not None and line_in_json['release_date'] != "" else '1500-01-01'

            formatted_data["revenue"] = int(
                line_in_json['revenue']) if line_in_json['revenue'] is not None else 0

            formatted_data["runtime"] = int(
                line_in_json['runtime']) if line_in_json['runtime'] is not None else 0

            formatted_data["adult"] = bool(
                line_in_json['adult']) if line_in_json['adult'] is not None else False

            formatted_data["backdrop_path"] = str(
                line_in_json['backdrop_path']) if line_in_json['backdrop_path'] is not any(variations_to_ignore) else None

            formatted_data["budget"] = int(
                line_in_json['budget']) if line_in_json['budget'] is not None else 0

            formatted_data["homepage"] = str(
                line_in_json['homepage']) if line_in_json['homepage'] is not any(variations_to_ignore) else None

            formatted_data["imdb_id"] = str(
                line_in_json['imdb_id']) if line_in_json['imdb_id'] is not any(variations_to_ignore) else None

            formatted_data["original_language"] = str(
                line_in_json['original_language']) if line_in_json['original_language'] is not any(variations_to_ignore) else None

            formatted_data["original_title"] = str(
                line_in_json['original_title']) if line_in_json['original_title'] is not any(variations_to_ignore) else None

            formatted_data["overview"] = str(
                line_in_json['overview']) if line_in_json['overview'] is not any(variations_to_ignore) else None

            formatted_data["popularity"] = float(
                line_in_json['popularity']) if line_in_json['popularity'] is not None else 0.0

            formatted_data["poster_path"] = str(
                line_in_json['poster_path']) if line_in_json['poster_path'] is not any(variations_to_ignore) else None

            formatted_data["tagline"] = str(
                line_in_json['tagline']) if line_in_json['tagline'] is not any(variations_to_ignore) else None
            
            formatted_data["keywords"] = str(line_in_json['keywords'])
            try:
                final_formatted_data = format_list_to_str(
                    line_in_json, formatted_data)
            except Exception as error:
                continue

            # remove and newline chracters
            updated_data = {}
            for key, value in final_formatted_data.items():
                if isinstance(value, str):  # Check if the value is a string
                    updated_value = value.replace("\n", " ").replace("\r", " ")
                    updated_data[key] = updated_value
                else:
                    updated_data[key] = value

            # add the data to our local list
            if updated_data['id'] not in unique_ids:
                unique_ids.add(updated_data['id'])
                data.append(updated_data)
    
    return data

## Merge the current data with the old dataset


In [36]:
def merger(logger: logging) -> None:
    global dataset_df
    logger.info('starting to merge new data with old dataframe')
    
    # Load and format the json data
    data = load_json_data()
    
    # Create a DataFrame from the loaded JSON data
    df = pd.DataFrame(data)
    df.replace({"Not Available": pd.NA,
               "1500-01-01": pd.NA,
               None:pd.NA}, inplace=True)
    
    # Concatenate the current dataset_df with the new df to merge the data
    merged_df = pd.concat([dataset_df, df])
    sorted_df = merged_df.sort_values('vote_count', ascending=False)

    # save the kaggle dataset
    sorted_df.to_csv(kaggle_file,
                     index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)

## Create the kaggle metadata file


In [37]:
def create_kaggle_info_file(logger: logging) -> None:
    logger.info('starting to create kaggle info file')
    
    # Define the data to be included in the kaggle info file
    data = {
        "id": "asaniczka/tmdb-movies-dataset-2023-930k-movies"
    }
    
    # the location of the metadata file
    metadata_file_location = os.path.join(
        kaggle_folder, 'dataset-metadata.json')

    # Open the metadata file in write mode and write the data as JSON
    with open(metadata_file_location, 'w', encoding='utf-8') as metadat_file:
        json.dump(data, metadat_file)


## Upload the dataset to kaggle


In [38]:
def upload_to_kaggle(logger) -> None:
    logger.info('starting to upload to kaggle')
    retries = 0
    while True and retries<5:
        try:
            command = f"kaggle datasets version -p '{kaggle_folder}' -m '{date_today} Update' -r zip"
            subprocess.run(command, shell=True, check=True)
            logger.info("Upload complete")
            break
        except Exception as error:
            logger.error(f"Error from Kaggle:{error}")
            time.sleep(5)
            retries+=1

In [39]:
def executor() -> None:
    logger = setup_logger()
    logger.info('Starting Executor')

    # first, lets get the id of the lastest movies
    latest = get_latest()

    # second, let's get the last movie from our last run
    oldest = get_oldest()
    
    # Generate a list of movie IDs
    movie_ids_list = list(range(oldest, latest))
    total_movies_to_process = len(movie_ids_list)
    
    logger.info(
        f"Total Movies to Process in this run: {total_movies_to_process}")
    delete_old_files(logger)
    
    with tqdm(total=total_movies_to_process, unit=' movies') as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = []
            for movie in movie_ids_list:
                futures.append(executor.submit(process_movie_ids, movie, logger, pbar))
            
            for future in concurrent.futures.as_completed(futures):
                try:
                    _ = future.result()
                except:
                    pass
    # combine all the threads to one file
    try:
        combine_threads(logger)
    except Exception as error:
        logger.critical(f"Error when combining threads: {error}")
        return
    
    # merge today's data with the old dataset
    try:
        merger(logger)
    except Exception as error:
        logger.critical(f"Error when merging dataframes {error}")
        return
    
    # create the kaggle metadata file
    try:
        create_kaggle_info_file(logger)
    except Exception as error:
        logger.critical(f"Error when creating kaggle info file {error}")
        return
    
    # upload the metadata file
    try:
        upload_to_kaggle(logger)
    except Exception as error:
        logger.critical(f"Error when creating kaggle info file {error}")
        return

    logger.info('Completed Executor')

# Let's run this


In [6]:
executor()

NameError: name 'executor' is not defined